In [1]:
from dolfin import *
import mshr
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML 
import time

In [2]:
def stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f, folder_name):     
    dt = T / num_steps # time step size
    t = dt
    mesh_f_s = []
    
    
    
    #input mesh choose
    if mesh_f == 0:
        mesh = Mesh('stenosis_f0.6.xml')
        boundary = MeshFunction('size_t', mesh,'stenosis_f0.6_facet_region.xml')
        mesh_f_s = 'R' #regular
    else:
        mesh = Mesh('stenosis_f0.6_fine.xml')
        boundary = MeshFunction('size_t', mesh,'stenosis_f0.6_fine_facet_region.xml')
        mesh_f_s = 'F' #fine
    h = CellDiameter(mesh)
    
    
    
    # Build function space
    P2 = VectorElement("P", mesh.ufl_cell(), element_n)
    P1 = FiniteElement("P", mesh.ufl_cell(), 1)
    TH = P2 * P1
    W = FunctionSpace(mesh, TH)
    u_par = Constant((0,0))
    
    
    
    #boundary
    ds = Measure('ds', domain=mesh, subdomain_data=boundary)
    n_v = FacetNormal(mesh)
    boundary_conditions = {1: {'Dirichlet': u_par},   #inflow
                           #2: {'Dirichlet': Constant(0)},   #outflow
                           #3: {'Dirichlet': Constant(0)},   #symmetry
                           4: {'Dirichlet': Constant((0.0, 0.0))}}   #no-slip
        # Collect Dirichlet conditions
    bc = DirichletBC(W.sub(0), boundary_conditions[1]['Dirichlet'], boundary, 1)
    
    

    # Define variational problem
    (u, p) = TrialFunctions(W)
    (v, q) = TestFunctions(W)
    u0 = project(Constant((0, 0)), VectorFunctionSpace(mesh, "P", 4))
    p0 = project(Constant(0), FunctionSpace(mesh, "P", 4))
    
    
    
    #f constant
    f = Constant((0, 0))
    f0 = Constant((0, 0))

    
    
    #file work
    folder =  folder_name + '/cn_' +str(conv_f)+ '-bc_' +str(bcflow_f)+ '-te_' +str(temam_f)+ '-su_' +str(supg_f)+ '-co_' +str(combi_f) + '_'+mesh_f_s+ '-dt_' + str(dt) + '-e_' + str(epsilon) + '-th_' + str(theta)  
    u_file = XDMFFile(folder + '/u.xdmf')
    p_file = XDMFFile(folder + '/p.xdmf')
    u_file.parameters['rewrite_function_mesh'] = False
    p_file.parameters['rewrite_function_mesh'] = False
    u_file.parameters["flush_output"] = True
    p_file.parameters["flush_output"] = True
    
    
    
    # Time-stepping
    proceent = 0
    t_arr = np.zeros(num_steps)
    
    
    
    #constant
    f = Constant((0, 0))
    f0 = Constant((0, 0))
    
    
    
    x, y = sym.symbols('x[0], x[1]')  
    u_par = (1.5 *Re *mu/2/rho/R)*(1 - (y/R)**2)
    u_D = sym.simplify(u_par)
    u_par_c = Expression((sym.printing.ccode(u_D), 0), degree=5)
    
    for n in range(num_steps):
        
        
        #progress
        if progress_f == 1:
            if int(n/(num_steps-1)*100) >= proceent:
                print("% =", int(n/(num_steps-1) * 100), ', time mplsd =', int((time.time()-t_start)))
                proceent+=proceents
        t_arr[n] = t
        
        
        #calculate u_bulk
        u_par = u_par_c*np.sin(np.pi*t/T)
        u_par = project(u_par, VectorFunctionSpace(mesh, "P", 4))
        bc_4 = DirichletBC(W.sub(0), u_par, boundary, 4)
        
        
        
        #boundary calculation
        bcs = []
        bcs.append(bc)
        bcs.append(bc_4)
        bc_3 = - inner(p0*n_v, v)*ds
        
        
        
        #Form parts calculation 
        a1 = (mu*inner(grad(u0), grad(v)) + div(v)*p0 + mu*q*div(u0))*dx(domain = mesh)
        L1 = inner(f0, v)*dx(domain = mesh)
        a2 = (mu*inner(grad(u), grad(v)) + div(v)*p + mu*q*div(u))*dx(domain = mesh) 
        L2 = inner(f, v)*dx(domain = mesh)
        
        
        
        
        #correction terms
        correction_t_r = temam_t_r = epsilon_t_r = conv_t_r = bcflow_t_r = supg_t_r = combi_t_r = temam_t_l = epsilon_t_l = conv_t_l = bcflow_t_l = supg_t_l = combi_t_l = 0
        
        
        if bcflow_f == 1:
            ala = inner(u0,n_v)
            #bcflow_t = bcflow_f * rho/4 * abs(ala - abs(ala)) * inner((theta*u - (1-theta)*u0), v) * dx(domain = mesh) 
            bcflow_t_l = bcflow_f * rho/4 * abs(ala - abs(ala)) * inner((theta*u), v) * ds
            bcflow_t_r = bcflow_f * rho/4 * abs(ala - abs(ala)) * inner(((1-theta)*u0), v) * ds
        else:
            bcflow_t_l = bcflow_t_r = 0
        
        
        
        if temam_f == 1:
            #temam_t = temam_f * rho/2 * inner(dot(grad(u0),(theta*u - (1-theta)*u0)), v)*dx(domain = mesh) 
            temam_t_l = temam_f * rho/2 * inner(dot(grad(u0),(theta*u)), v)*dx(domain = mesh)
            temam_t_r = temam_f * rho/2 * inner(dot(grad(u0),((1-theta)*u0)), v)*dx(domain = mesh)                                   
        else:
            temam_f_t_l = temam_f_t_r = 0
        
        
        
        if epsilon == 1:
            #epsilon_t =  epsilon *h*h/mu*inner(grad(q), grad((theta*p - (1-theta)*p0)))*dx(domain = mesh)
            epsilon_t_l =  epsilon *h*h/mu*inner(grad(q), grad(theta*p))*dx(domain = mesh) 
            epsilon_t_r =  epsilon *h*h/mu*inner(grad(q), (1-theta) * grad(p0))*dx(domain = mesh)
        else:
            epsilon_t_l = epsilon_t_r = 0
        
        
        
        if conv_f == 1:                              
            #conv_t =  conv_f * rho*inner(dot(u0, nabla_grad((theta*u - (1-theta)*u0))), v) * dx(domain = mesh) 
            conv_t_l =  conv_f * rho*inner(dot(u0, nabla_grad((theta*u))), v) * dx(domain = mesh)
            conv_t_r =  conv_f * rho*inner(dot(u0, (1-theta)*nabla_grad((u0))), v) * dx(domain = mesh)                                    
        else:
            conv_t_l = conv_t_r = 0
        
        
        
        if supg_f == 1:                              
            supg_t_ =  supg_f * (4/dt/dt + 4*dot(u0,u0)/h/h + (12*mu/rho/h/h)**2)**(-0.5)
            #supg_t =  supg_t_ * rho * inner(dot(u0, nabla_grad((theta*u - (1-theta)*u0))), rho * dot(u0, nabla_grad(v))) * dx(domain = mesh)
            supg_t_l =  supg_t_ * rho * inner(dot(u0, nabla_grad((theta*u))), rho * dot(u0, nabla_grad(v))) * dx(domain = mesh)                                     
            supg_t_r =  supg_t_ * rho * inner(dot(u0, (1-theta)*nabla_grad((u0))), rho * dot(u0, nabla_grad(v))) * dx(domain = mesh)                                                                    
        else:
            supg_t_l = supg_t_r = 0
        
        
        
        if combi_f == 1:
            supg_t_ =  supg_f * (4/dt/dt + 4*dot(u0,u0)/h/h + (12*mu/rho/h/h)**2)**(-0.5)
            #combi_t =  combi_f * supg_t_ * inner((rho * dot(u0, nabla_grad((theta*u - (1-theta)*u0))) + grad((theta*p - (1-theta)*p0))),(rho * dot(u0, nabla_grad(v)) - grad(q))) * dx(domain = mesh)  
            combi_t_l =  combi_f * supg_t_ * inner((rho * dot(u0, nabla_grad((theta*u))) + grad((theta*p))),(rho * dot(u0, nabla_grad(v)) - grad(q))) * dx(domain = mesh) 
            combi_t_r =  combi_f * supg_t_ * inner((rho * dot(u0, (1-theta)*nabla_grad((u0))) + (1-theta)*grad((p0))),(rho * dot(u0, nabla_grad(v)) - grad(q))) * dx(domain = mesh) 
        else:
            combi_t_l = combi_t_r = 0
        
        
                                            
        #Form calculation 
        correction_t_r = temam_t_r + epsilon_t_r + conv_t_r + bcflow_t_r + supg_t_r + combi_t_r +bc_3
        correction_t_l = temam_t_l + epsilon_t_l + conv_t_l + bcflow_t_l + supg_t_l + combi_t_l
        
        
        a = inner(u,v) * dx + (theta) * (a2) + correction_t_l
        L =  (1 - theta) * (a1 - L1) - (theta) * L2 - inner(u0,v)*dx + correction_t_r
        U = Function(W)
        solve(a == L, U, bcs)
        u0, p0 = U.split()
        
        
        
        #file work
        u0.rename("v0", "velocity")
        p0.rename("p0", "pressure")
        u_file.write(u0, t)
        p_file.write(p0, t)
        
        
        
        #time step apply
        t += dt
    
    
    
    #file close
    u_file.close()
    p_file.close()
    if plot_f == 1:
        plt.plot(t_arr,vc_arr)

2.4

Solve now the Navier-Stokes problem using $\mathbb{P}_{1} / \mathbb{P}_{1}$ with a standard pressure stabilization
$$
\varepsilon \frac{h^{2}}{\mu} \int_{\Omega} \nabla p^{k+\theta} \cdot \nabla q
$$
with $h$ the cell diameter $(\mathrm{h}=$ CellDiameter (mesh) $)$. Employ the backflow and Temam stabilization. Perform simulations using $\varepsilon=0.01$ and $\varepsilon=0.0001$ for $R e=5000$ for the fine mesh. As before, $\theta=1, \tau=0.01$.

2.4.1. Assignment (0.5 points) Prove that the fully discrete solution is asymptotically stable for $\theta \geq 1 / 2$

We will start with the form obtained in the 6.2.1 part. 

$\rho \int_{\Omega} \dot{\vec{u_{n}}} \cdot \vec{u_{n}}+\rho \int_{\Omega}(\vec{u_{n}} \cdot \nabla) \vec{u_{n}} \cdot \vec{u_{n}}-\int_{\Omega} \mu_{n} \Delta \vec{u_{n}} \cdot \vec{u_{n}}$
$+\int_{\Omega} \nabla p \cdot \vec{u_{n}}=\int_{\Omega} \vec{f} \cdot \vec{u_{n}}$

and change it to add the $\theta$ - scheme with semi-implict stabilization of $\vec{u_n^c} = \vec{u^k}$

$\rho \int_{\Omega} \frac{\vec{u}_{n}^{k+1}-\int_{\Omega}\vec{u}_{n}^{k}}{\tau} \cdot \vec{v}_{n}+\int_{\Omega}\frac{\rho}{2}\left(\nabla \cdot \vec{u}_{n}^{k}\right) \vec{u}_{n}^{k+\theta} \cdot \vec{v}_{n}+\int_{\Omega}\mu \nabla \vec{u}_{n}^{k+\theta} \cdot \vec{v}_n -\int_{\Omega}p_{n}^{k+\theta} \nabla \cdot \vec{v}_{n}+\int_{\Omega}g_{n} \nabla \cdot \vec{u}_{n}^{k+\theta}=\int_{\Omega} \vec{f}^{k + \theta} \cdot v_{n}$

Then we add backflow stabilization 
$
C_{b a c k} \int_{\Gamma_{N}}\left| \vec{u}^{k} \cdot \vec{n}\right|_{-} \vec{u}^{k+\theta} \cdot \vec{v}
$

with
$
|c|_{-}:=\frac{1}{2} \operatorname{abs}(c-\operatorname{abs}(c))
$

add Teman stabilization 

$\frac{\rho}{2} \int_{\Omega}\left(\nabla \cdot \vec{u}^{k}\right) \vec{u}^{k+\theta} \cdot \vec{v}$



And finally add the pressure stabilization 

$\varepsilon \frac{h^{2}}{\mu} \int_{\Omega} \nabla p^{k+\theta} \cdot \nabla q$

And we get this weak form
$\rho \int_{\Omega} \frac{\vec{u}_{n}^{k+1}-\vec{u}_{n}^{k}}{\tau} \cdot \vec{v}_{n} + \int_{\Omega}\frac{\rho}{2}\left(\nabla \cdot \vec{u}_{n}^{k}\right) \vec{u}_{n}^{k+\theta} \cdot \vec{v}_{n}+\int_{\Omega}\mu \nabla \vec{u}_{n}^{k+\theta} \cdot \vec{v}_n -\int_{\Omega}p_{n}^{k+\theta} \nabla \cdot \vec{v}_{n}+\int_{\Omega}g_{n} \nabla \cdot \vec{u}_{n}^{k+\theta} - \frac{\rho}{2} \int_{\Omega}\left(\nabla \cdot \vec{u}^{k}\right) \vec{u}^{k+\theta} \cdot \vec{v}  + \varepsilon \frac{h^{2}}{\mu} \int_{\Omega} \nabla p^{k+\theta} \cdot \nabla q + C_{b a c k} \int_{\Gamma_{N}}\left| \vec{u}^{k} \cdot \vec{n}\right|_{-} \vec{u}^{k+\theta} \cdot \vec{v} =\int_{\Omega} \vec{f}^{k + \theta} \cdot v_{n}$


Algebra skills implemented we get rid of this term $\int_{\Omega}\frac{\rho}{2}\left(\nabla \cdot \vec{u}_{n}^{k}\right) \vec{u}_{n}^{k+\theta} \cdot \vec{v}_{n}$


$\rho \int_{\Omega} \frac{\vec{u}_{n}^{k+1}-\vec{u}_{n}^{k}}{\tau} \cdot \vec{v}_{n} +\int_{\Omega}\mu \nabla \vec{u}_{n}^{k+\theta} \cdot \vec{v}_n -\int_{\Omega}p_{n}^{k+\theta} \nabla \cdot \vec{v}_{n}+\int_{\Omega}g_{n} \nabla \cdot \vec{u}_{n}^{k+\theta}  + \varepsilon \frac{h^{2}}{\mu} \int_{\Omega} \nabla p^{k+\theta} \cdot \nabla q + C_{b a c k} \int_{\Gamma_{N}}\left| \vec{u}^{k} \cdot \vec{n}\right|_{-} \vec{u}^{k+\theta} \cdot \vec{v} =\int_{\Omega} \vec{f}^{k + \theta} \cdot v_{n}$


Rewrite the equation

$\rho \int_{\Omega} \frac{\vec{u}_{n}^{k+1}-\vec{u}_{n}^{k}}{\tau} \cdot \vec{v}_{n} = -\int_{\Omega}\mu \nabla \vec{u}_{n}^{k+\theta} \cdot \vec{v}_n +\int_{\Omega}p_{n}^{k+\theta} \nabla \cdot \vec{v}_{n}-\int_{\Omega}g_{n} \nabla \cdot \vec{u}_{n}^{k+\theta}  - \varepsilon \frac{h^{2}}{\mu} \int_{\Omega} \nabla p^{k+\theta} \cdot \nabla q - C_{b a c k} \int_{\Gamma_{N}}\left| \vec{u}^{k} \cdot \vec{n}\right|_{-} \vec{u}^{k+\theta} \cdot \vec{v} + \int_{\Omega} \vec{f}^{k + \theta} \cdot v_{n}$

so when we deal with energy
$-\frac{\rho}{2 \tau} \int_{\Omega}\left\|\vec{u}_{h}^{k}\right\|^{2}+\frac{\rho}{2 \tau} \int_{\Omega}\left\|\vec{u}_{n}^{k+1}\right\|^{2}= \left(\theta-\frac{1}{2}\right) \rho \int_{\Omega}\left\|\vec{u}_{n}^{k+1}-\vec{u}_{u}^{k}\right\|^{2} + \mu \int_{\Omega} \left\|\nabla u_{n}^{k+\theta} \right\|^{2}$

and all the other terms are negative
so we have unconditional stability for $\theta \geq \frac{1}{2}$ as the energy decreases with the bigger timestep

2.4.2. Assignment (1 point) Compare the numerical results for both values of $\varepsilon$ to the results for $\mathbb{P}_{2} / \mathbb{P}_{1}$ (without the stabilization term (4)). Comment on the differences and similarities between all three cases.

First coarse mesh

Re = 5000 no stabilization term

In [4]:
folder_name = '6.2.4/6.2.4.2/coarse'

t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0
theta = 1
R = 1
element_n = 2 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 0 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 1
temam_f = 1
supg_f = 0
combi_f = 0


plot_f = 0

Re = 5000

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f ,folder_name)

% = 0 , time mplsd = 2
% = 10 , time mplsd = 40
% = 20 , time mplsd = 79
% = 30 , time mplsd = 117
% = 40 , time mplsd = 156
% = 50 , time mplsd = 195
% = 60 , time mplsd = 235
% = 70 , time mplsd = 274
% = 80 , time mplsd = 314
% = 90 , time mplsd = 354
% = 100 , time mplsd = 393


Re = 5000 epsilon = 0,01

In [5]:
t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0.01
theta = 1
R = 1
element_n = 2 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 0 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 1
temam_f = 1
supg_f = 0
combi_f = 0


plot_f = 0

Re = 5000

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f ,folder_name)

% = 0 , time mplsd = 1
% = 10 , time mplsd = 40
% = 20 , time mplsd = 79
% = 30 , time mplsd = 117
% = 40 , time mplsd = 156
% = 50 , time mplsd = 195
% = 60 , time mplsd = 235
% = 70 , time mplsd = 274
% = 80 , time mplsd = 314
% = 90 , time mplsd = 354
% = 100 , time mplsd = 393


Re = 5000 epsilon = 0,0001

In [6]:
t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0.0001
theta = 1
R = 1
element_n = 2 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 0 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 1
temam_f = 1
supg_f = 0
combi_f = 0


plot_f = 0

Re = 5000

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f ,folder_name)

% = 0 , time mplsd = 0
% = 10 , time mplsd = 39
% = 20 , time mplsd = 77
% = 30 , time mplsd = 115
% = 40 , time mplsd = 154
% = 50 , time mplsd = 195
% = 60 , time mplsd = 234
% = 70 , time mplsd = 274
% = 80 , time mplsd = 314
% = 90 , time mplsd = 355
% = 100 , time mplsd = 394


Second fine mesh

Re = 5000 no stabilization term

In [7]:
folder_name = '6.2.4/6.2.4.2/fine'
t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0
theta = 1
R = 1
element_n = 2 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 1 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 1
temam_f = 1
supg_f = 0
combi_f = 0


plot_f = 0

Re = 5000

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f ,folder_name)

% = 0 , time mplsd = 4
% = 10 , time mplsd = 190
% = 20 , time mplsd = 374
% = 30 , time mplsd = 1165
% = 40 , time mplsd = 1357
% = 50 , time mplsd = 1546
% = 60 , time mplsd = 1745
% = 70 , time mplsd = 1942
% = 80 , time mplsd = 2141
% = 90 , time mplsd = 2340
% = 100 , time mplsd = 2531


Re = 5000 epsilon = 0,01

In [8]:
t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0.01
theta = 1
R = 1
element_n = 2 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 1 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 1
temam_f = 1
supg_f = 0
combi_f = 0


plot_f = 0

Re = 5000

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f ,folder_name)

% = 0 , time mplsd = 4
% = 10 , time mplsd = 202
% = 20 , time mplsd = 398
% = 30 , time mplsd = 595
% = 40 , time mplsd = 782
% = 50 , time mplsd = 978
% = 60 , time mplsd = 1168
% = 70 , time mplsd = 1356
% = 80 , time mplsd = 1557
% = 90 , time mplsd = 1754
% = 100 , time mplsd = 1949


Re = 5000 epsilon = 0,0001

In [10]:
t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0.0001
theta = 1
R = 1
element_n = 2 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 1 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 1
temam_f = 1
supg_f = 0
combi_f = 0


plot_f = 0

Re = 5000

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f ,folder_name)

% = 0 , time mplsd = 4
% = 10 , time mplsd = 196
% = 20 , time mplsd = 389
% = 30 , time mplsd = 583
% = 40 , time mplsd = 777
% = 50 , time mplsd = 975
% = 60 , time mplsd = 1166
% = 70 , time mplsd = 1359
% = 80 , time mplsd = 1548
% = 90 , time mplsd = 1742
% = 100 , time mplsd = 1925


2.4.3. Assignment (0.5 points) Compute the previous case with $\theta=1 / 2, \mathbb{P}_{1} / \mathbb{P}_{1}$, $\varepsilon=0.0001$. Compare the results with the case $\theta=1$. Is the solution you obtained with $\theta=1 / 2$ what you would expected from the physics? If not, what if you turn off all convection-related terms for $\theta=0.5 ?$

In [ ]:
First coarse mesh

In [11]:
folder_name = '6.2.4/6.2.4.3/coarse'

t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0.0001
theta = 0.5
R = 1
element_n = 1 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 0 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 1
temam_f = 1
supg_f = 0
combi_f = 0


plot_f = 0

Re = 5000

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f ,folder_name)

% = 0 , time mplsd = 0
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
% = 10 , time mplsd = 35
% = 20 , time mplsd = 68
% = 30 , time mplsd = 101
% = 40 , time mplsd = 132
% = 50 , time mplsd = 164
% = 60 , time mplsd = 197
% = 70 , time mplsd = 228
% = 80 , time mplsd = 260
% = 90 , time mplsd = 292
% = 100 , time mplsd = 323


theta = 1

In [12]:
t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0.0001
theta = 1
R = 1
element_n = 1 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 0 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 1
temam_f = 1
supg_f = 0
combi_f = 0


plot_f = 0

Re = 5000

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f ,folder_name)

% = 0 , time mplsd = 0
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
% = 10 , time mplsd = 33
% = 20 , time mplsd = 64
% = 30 , time mplsd = 96
% = 40 , time mplsd = 127
% = 50 , time mplsd = 158
% = 60 , time mplsd = 190
% = 70 , time mplsd = 222
% = 80 , time mplsd = 254
% = 90 , time mplsd = 286
% = 100 , time mplsd = 317


In [ ]:
Second fine mesh

In [13]:
folder_name = '6.2.4/6.2.4.3/fine'

t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0.0001
theta = 0.5
R = 1
element_n = 1 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 1 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 1
temam_f = 1
supg_f = 0
combi_f = 0


plot_f = 0

Re = 5000

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f ,folder_name)

% = 0 , time mplsd = 4
% = 10 , time mplsd = 152
% = 20 , time mplsd = 301
% = 30 , time mplsd = 453
% = 40 , time mplsd = 1367
% = 50 , time mplsd = 1521
% = 60 , time mplsd = 1690
% = 70 , time mplsd = 1851
% = 80 , time mplsd = 2012
% = 90 , time mplsd = 2159
% = 100 , time mplsd = 2305


theta = 1

In [ ]:
folder_name = '6.2.4/6.2.4.3/fine'
t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0.0001
theta = 1
R = 1
element_n = 1 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 1 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 1
temam_f = 1
supg_f = 0
combi_f = 0


plot_f = 0

Re = 5000

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f ,folder_name)

% = 0 , time mplsd = 4
% = 10 , time mplsd = 166
% = 20 , time mplsd = 327
% = 30 , time mplsd = 493
% = 40 , time mplsd = 661
